**Data Processing for Car-Dheko Used Car Price Prediction**

In [28]:
import pandas as pd

file_path = r"C:\Users\USER\Desktop\Car-Dheko_Used_Car_Price_Prediction\csv_files\bangalore_cars.xlsx"

# Get all sheet names
sheets = pd.ExcelFile(file_path).sheet_names
print("Available sheets:", sheets)


Available sheets: ['bangalore_cars.csv']


In [31]:
df1 = pd.read_excel(file_path, sheet_name="bangalore_cars.csv")  # or sheet_name=0 for the first sheet
print(df1.head())

                                      new_car_detail  \
0  {'it': 0, 'ft': 'Petrol', 'bt': 'Hatchback', '...   
1  {'it': 0, 'ft': 'Petrol', 'bt': 'SUV', 'km': '...   
2  {'it': 0, 'ft': 'Petrol', 'bt': 'Hatchback', '...   
3  {'it': 0, 'ft': 'Petrol', 'bt': 'Sedan', 'km':...   
4  {'it': 0, 'ft': 'Diesel', 'bt': 'SUV', 'km': '...   

                                    new_car_overview  \
0  {'heading': 'Car overview', 'top': [{'key': 'R...   
1  {'heading': 'Car overview', 'top': [{'key': 'R...   
2  {'heading': 'Car overview', 'top': [{'key': 'R...   
3  {'heading': 'Car overview', 'top': [{'key': 'R...   
4  {'heading': 'Car overview', 'top': [{'key': 'R...   

                                     new_car_feature  \
0  {'heading': 'Features', 'top': [{'value': 'Pow...   
1  {'heading': 'Features', 'top': [{'value': 'Pow...   
2  {'heading': 'Features', 'top': [{'value': 'Pow...   
3  {'heading': 'Features', 'top': [{'value': 'Pow...   
4  {'heading': 'Features', 'top': [{'value': '

Method 1: Display Full Dictionary (No Truncation)

In [38]:
print(df1.columns.tolist())

['car_links', 'detail_it', 'detail_ft', 'detail_bt', 'detail_km', 'detail_transmission', 'detail_ownerNo', 'detail_owner', 'detail_oem', 'detail_model', 'detail_modelYear', 'detail_centralVariantId', 'detail_variantName', 'detail_price', 'detail_priceActual', 'detail_priceSaving', 'detail_priceFixedText', 'detail_trendingText.imgUrl', 'detail_trendingText.heading', 'detail_trendingText.desc', 'new_car_overview_heading', 'new_car_overview_top', 'new_car_overview_bottomData', 'new_car_feature_heading', 'new_car_feature_top', 'new_car_feature_data', 'new_car_feature_commonIcon', 'new_car_specs_heading', 'new_car_specs_top', 'new_car_specs_data', 'new_car_specs_commonIcon', 'first_feature']


In [56]:
# Set Pandas to show full content of dictionaries
pd.set_option('display.max_colwidth', None)  # Disable column width truncation

# Display the 'new_car_detail' column
print(df1['new_car_overview_bottomData'].apply(lambda x: x))

0       None
1       None
2       None
3       None
4       None
5       None
6       None
7       None
8       None
9       None
10      None
11      None
12      None
13      None
14      None
15      None
16      None
17      None
18      None
19      None
20      None
21      None
22      None
23      None
24      None
25      None
26      None
27      None
28      None
29      None
30      None
31      None
32      None
33      None
34      None
35      None
36      None
37      None
38      None
39      None
40      None
41      None
42      None
43      None
44      None
45      None
46      None
47      None
48      None
49      None
50      None
51      None
52      None
53      None
54      None
55      None
56      None
57      None
58      None
59      None
60      None
61      None
62      None
63      None
64      None
65      None
66      None
67      None
68      None
69      None
70      None
71      None
72      None
73      None
74      None
75      None
76      None

In [54]:
print(df1.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1481 entries, 0 to 1480
Data columns (total 32 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   car_links                    1481 non-null   object
 1   detail_it                    1481 non-null   int64 
 2   detail_ft                    1481 non-null   object
 3   detail_bt                    1481 non-null   object
 4   detail_km                    1481 non-null   object
 5   detail_transmission          1481 non-null   object
 6   detail_ownerNo               1481 non-null   int64 
 7   detail_owner                 1481 non-null   object
 8   detail_oem                   1481 non-null   object
 9   detail_model                 1481 non-null   object
 10  detail_modelYear             1481 non-null   int64 
 11  detail_centralVariantId      1481 non-null   int64 
 12  detail_variantName           1481 non-null   object
 13  detail_price                 1481

Step 1: Convert JSON strings to Python dictionaries
First, ensure the columns contain actual dictionaries (not strings):

In [32]:
import ast

# Convert stringified JSON to dict (if needed)
for col in ['new_car_detail', 'new_car_overview', 'new_car_feature', 'new_car_specs']:
    if isinstance(df1[col].iloc[0], str):
        df1[col] = df1[col].apply(ast.literal_eval)

Step 2: Flatten nested columns
We'll use json_normalize to unpack dictionaries into separate columns:

In [33]:
from pandas import json_normalize

# Flatten the 'new_car_detail' column
detail_cols = json_normalize(df1['new_car_detail'])
detail_cols.columns = [f"detail_{col}" for col in detail_cols.columns]
df1 = pd.concat([df1, detail_cols], axis=1).drop('new_car_detail', axis=1)

B. Flatten other nested columns
Repeat for other columns (adjust field names as needed):

In [34]:
for col in ['new_car_overview', 'new_car_feature', 'new_car_specs']:
    flattened = json_normalize(df1[col])
    flattened.columns = [f"{col}_{subcol}" for subcol in flattened.columns]
    df1 = pd.concat([df1, flattened], axis=1).drop(col, axis=1)

Step 3: View ALL Expanded Data
To see all rows and columns without truncation:

In [35]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 50)  # Adjust column width

print(df1.head())  # Now shows full expanded data

                                           car_links  detail_it detail_ft  detail_bt detail_km detail_transmission  detail_ownerNo detail_owner detail_oem        detail_model  detail_modelYear  detail_centralVariantId        detail_variantName detail_price detail_priceActual detail_priceSaving detail_priceFixedText                         detail_trendingText.imgUrl detail_trendingText.heading             detail_trendingText.desc new_car_overview_heading                               new_car_overview_top new_car_overview_bottomData new_car_feature_heading                                new_car_feature_top                               new_car_feature_data                         new_car_feature_commonIcon new_car_specs_heading                                  new_car_specs_top                                 new_car_specs_data new_car_specs_commonIcon
0  https://www.cardekho.com/used-car-details/used...          0    Petrol  Hatchback  1,20,000              Manual               3    3rd

In [36]:
# Example: Extract first feature
df1['first_feature'] = df1['new_car_feature_top'].apply(lambda x: x[0]['value'] if x else None)

work

In [2]:
import pandas as pd
import ast
from pandas import json_normalize

# Load the Excel file
file_path = "csv_files/bangalore_cars.xlsx"
df = pd.read_excel(file_path, sheet_name="bangalore_cars.csv")

# Function to convert string to dictionary
def parse_dict_column(column):
    return column.apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# Parse each dictionary column
df["new_car_detail"] = parse_dict_column(df["new_car_detail"])
df["new_car_overview"] = parse_dict_column(df["new_car_overview"])
df["new_car_feature"] = parse_dict_column(df["new_car_feature"])
df["new_car_specs"] = parse_dict_column(df["new_car_specs"])

# Normalize and flatten each dictionary
df_banglore_detail = json_normalize(df["new_car_detail"])
df_banglore_overview = json_normalize(df["new_car_overview"].apply(lambda x: {item['key']: item['value'] for item in x.get('top', [])}))
df_banglore_feature = json_normalize(df["new_car_feature"].apply(lambda x: {'Feature_' + str(i): f['value'] for i, f in enumerate(x.get('top', []))}))
df_banglore_specs = json_normalize(df["new_car_specs"].apply(lambda x: {item['key']: item['value'] for item in x.get('top', [])}))

# Add city name
for d in [df_banglore_detail, df_banglore_overview, df_banglore_feature, df_banglore_specs]:
    d["City"] = "bangalore"

# Combine all structured data
df_final = pd.concat([df_banglore_detail, df_banglore_overview, df_banglore_feature, df_banglore_specs], axis=1)

# Save to Excel
df_final.to_excel("structured_bangalore_cars.xlsx", index=False)

print("Data has been structured and saved to 'structured_bangalore_cars.xlsx'")


Data has been structured and saved to 'structured_bangalore_cars.xlsx'


work2

In [7]:
import pandas as pd
import ast
from pandas import json_normalize

# Load the Excel file
file_path = "csv_files/bangalore_cars.xlsx"
df = pd.read_excel(file_path, sheet_name="bangalore_cars.csv")

# Function to convert string to dictionary
def parse_dict_column(column):
    return column.apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# Parse dictionary-like columns
df["new_car_detail"] = parse_dict_column(df["new_car_detail"])
df["new_car_overview"] = parse_dict_column(df["new_car_overview"])
df["new_car_feature"] = parse_dict_column(df["new_car_feature"])
df["new_car_specs"] = parse_dict_column(df["new_car_specs"])

# --- Detail flattening ---
df_banglore_detail = json_normalize(df["new_car_detail"])

# --- Overview flattening ---
def flatten_overview(overview_data):
    return {item['key']: item['value'] for item in overview_data.get("top", []) if isinstance(item, dict)}

df_banglore_overview = pd.json_normalize(df["new_car_overview"].apply(flatten_overview))

# --- Feature flattening (flatten all nested lists) ---
def flatten_feature_list(feature_data):
    all_features = []
    if isinstance(feature_data, dict):
        for section in feature_data.values():  # loop over 'top', 'others', etc.
            if isinstance(section, list):
                all_features.extend(section)
    return {'Feature_' + str(i): f.get('value') for i, f in enumerate(all_features)}

df_banglore_feature = pd.json_normalize(df["new_car_feature"].apply(flatten_feature_list))

# --- Specs flattening (flatten all nested lists) ---
def flatten_specs_list(specs_data):
    all_specs = []
    if isinstance(specs_data, dict):
        for section in specs_data.values():
            if isinstance(section, list):
                all_specs.extend(section)
    return {spec.get('key'): spec.get('value') for spec in all_specs if isinstance(spec, dict) and 'key' in spec and 'value' in spec}

df_banglore_specs = pd.json_normalize(df["new_car_specs"].apply(flatten_specs_list))

# --- Combine all sections ---
df_final = pd.concat([df_banglore_detail, df_banglore_overview, df_banglore_feature, df_banglore_specs], axis=1)

# --- Add city only once ---
df_final["City"] = "bangalore"

# --- Save to Excel ---
df_final.to_excel("structured_bangalore_cars.xlsx", index=False)

print("✅ Data has been structured and saved to 'structured_bangalore_cars.xlsx'")


✅ Data has been structured and saved to 'structured_bangalore_cars.xlsx'


work 3

In [12]:
import pandas as pd
import ast
from pandas import json_normalize

# Load the Excel file
file_path = "csv_files/bangalore_cars.xlsx"
df = pd.read_excel(file_path, sheet_name="bangalore_cars.csv")

# Convert stringified dictionaries to actual dicts
def parse_dict_column(column):
    return column.apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

df["new_car_detail"] = parse_dict_column(df["new_car_detail"])
df["new_car_overview"] = parse_dict_column(df["new_car_overview"])
df["new_car_feature"] = parse_dict_column(df["new_car_feature"])
df["new_car_specs"] = parse_dict_column(df["new_car_specs"])

# ------------------------
# OVERVIEW
# ------------------------
def flatten_overview(overview_data):
    if isinstance(overview_data, dict):
        return {item['key']: item['value'] for item in overview_data.get("top", []) if isinstance(item, dict)}
    return {}

df_banglore_overview = pd.json_normalize(df["new_car_overview"].apply(flatten_overview))

# ------------------------
# DETAIL
# ------------------------
df_banglore_detail = json_normalize(df["new_car_detail"])

# ------------------------
# FEATURE: top + data[*].list
# ------------------------
def flatten_feature_all(data):
    features = []
    
    # Collect top values
    if isinstance(data, dict):
        top = data.get("top", [])
        features.extend(f.get("value") for f in top if isinstance(f, dict))
        
        # Collect from data section
        for section in data.get("data", []):
            if isinstance(section, dict):
                for item in section.get("list", []):
                    if isinstance(item, dict):
                        val = item.get("value")
                        if val:
                            features.append(val)

    # Return as key-value Feature_0, Feature_1...
    return {'Feature_' + str(i): v for i, v in enumerate(features)}

df_banglore_feature = pd.json_normalize(df["new_car_feature"].apply(flatten_feature_all))

# ------------------------
# SPECS: top + data[*].list (key-value pairs)
# ------------------------
def flatten_specs_all(data):
    specs = {}

    if isinstance(data, dict):
        # Top section
        for item in data.get("top", []):
            if isinstance(item, dict):
                k = item.get("key")
                v = item.get("value")
                if k and v:
                    specs[k] = v

        # From data -> list
        for section in data.get("data", []):
            if isinstance(section, dict):
                for item in section.get("list", []):
                    if isinstance(item, dict):
                        k = item.get("key")
                        v = item.get("value")
                        if k and v:
                            specs[k] = v
    return specs

df_banglore_specs = pd.json_normalize(df["new_car_specs"].apply(flatten_specs_all))

# ------------------------
# COMBINE ALL & ADD CITY ONCE
# ------------------------
df_final = pd.concat([df_banglore_detail, df_banglore_overview, df_banglore_feature, df_banglore_specs], axis=1)
df_final["City"] = "bangalore"

# ------------------------
# SAVE TO EXCEL
# ------------------------
df_final.to_excel("structured_bangalore_cars.xlsx", index=False)

print("✅ All nested features and specs parsed successfully and saved to 'structured_bangalore_cars.xlsx'")


✅ All nested features and specs parsed successfully and saved to 'structured_bangalore_cars.xlsx'


In [8]:
import pandas as pd

file_path = r"C:\Users\USER\Desktop\Car-Dheko_Used_Car_Price_Prediction\structured_bangalore_cars.xlsx"

# Get all sheet names
sheets = pd.ExcelFile(file_path).sheet_names
print("Available sheets:", sheets)


Available sheets: ['Sheet1']


In [9]:
df1 = pd.read_excel(file_path, sheet_name="Sheet1")  # or sheet_name=0 for the first sheet
print(df1.head())

   it      ft         bt        km transmission  ownerNo      owner      oem  \
0   0  Petrol  Hatchback  1,20,000       Manual        3  3rd Owner   Maruti   
1   0  Petrol        SUV    32,706       Manual        2  2nd Owner     Ford   
2   0  Petrol  Hatchback    11,949       Manual        1  1st Owner     Tata   
3   0  Petrol      Sedan    17,794       Manual        1  1st Owner  Hyundai   
4   0  Diesel        SUV    60,000       Manual        1  1st Owner   Maruti   

                model  modelYear  ...  Feature_11 Feature_12 Feature_13  \
0      Maruti Celerio       2015  ...         NaN        NaN        NaN   
1       Ford Ecosport       2018  ...         NaN        NaN        NaN   
2          Tata Tiago       2018  ...         NaN        NaN        NaN   
3       Hyundai Xcent       2014  ...         NaN        NaN        NaN   
4  Maruti SX4 S Cross       2015  ...         NaN        NaN        NaN   

      Mileage   Engine  Max Power    Torque Seats.1 Wheel Size      

In [10]:
print(df1.columns.tolist())

['it', 'ft', 'bt', 'km', 'transmission', 'ownerNo', 'owner', 'oem', 'model', 'modelYear', 'centralVariantId', 'variantName', 'price', 'priceActual', 'priceSaving', 'priceFixedText', 'trendingText.imgUrl', 'trendingText.heading', 'trendingText.desc', 'Registration Year', 'Insurance Validity', 'Fuel Type', 'Seats', 'Kms Driven', 'RTO', 'Ownership', 'Engine Displacement', 'Transmission', 'Year of Manufacture', 'Feature_0', 'Feature_1', 'Feature_2', 'Feature_3', 'Feature_4', 'Feature_5', 'Feature_6', 'Feature_7', 'Feature_8', 'Feature_9', 'Feature_10', 'Feature_11', 'Feature_12', 'Feature_13', 'Mileage', 'Engine', 'Max Power', 'Torque', 'Seats.1', 'Wheel Size', 'City']


In [11]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1481 entries, 0 to 1480
Data columns (total 50 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   it                    1481 non-null   int64  
 1   ft                    1481 non-null   object 
 2   bt                    1481 non-null   object 
 3   km                    1481 non-null   object 
 4   transmission          1481 non-null   object 
 5   ownerNo               1481 non-null   int64  
 6   owner                 1481 non-null   object 
 7   oem                   1481 non-null   object 
 8   model                 1481 non-null   object 
 9   modelYear             1481 non-null   int64  
 10  centralVariantId      1481 non-null   int64  
 11  variantName           1481 non-null   object 
 12  price                 1481 non-null   object 
 13  priceActual           311 non-null    object 
 14  priceSaving           0 non-null      float64
 15  priceFixedText       

In [1]:
import pandas as pd
import os
from glob import glob

# 1. Specify the correct folder path (where your Excel files are stored)
folder_path = "csv_files"  # Replace with your actual folder name if different

# 2. List all city Excel files in the folder
city_files = glob(os.path.join(folder_path, '*.xlsx'))  # Searches inside 'csv_files' folder
print(f"Found Excel files in '{folder_path}':", city_files)

if not city_files:
    print(f"\n❌ No Excel files found in '{folder_path}'. Please check:")
    print("- Folder name is correct (case-sensitive)")
    print("- Files have '.xlsx' extension")
    print("- Files are not in a sub-subfolder")
else:
    # 3. Create a summary DataFrame
    summary_data = []

    for file in city_files:
        city_name = os.path.splitext(os.path.basename(file))[0]  # Extract filename without path/extension
        try:
            # Read first 2 rows to check structure
            df_sample = pd.read_excel(file, nrows=2)
            
            # Get file info
            file_size = f"{round(os.path.getsize(file)/(1024*1024), 2)} MB"
            file_columns = df_sample.columns.tolist()
            file_shape = pd.read_excel(file).shape
            
            summary_data.append({
                'City': city_name,
                'File_Type': 'Excel (XLSX)',
                'File_Size': file_size,
                'Rows': file_shape[0],
                'Columns': file_shape[1],
                'Columns_List': file_columns,
                'Sample_First_Row': df_sample.iloc[0].to_dict() if len(df_sample) > 0 else {}
            })
        except Exception as e:
            print(f"Error reading {file}: {str(e)}")
            summary_data.append({
                'City': city_name,
                'Error': str(e)
            })

    # 4. Create summary DataFrame
    summary_df = pd.DataFrame(summary_data)

    # 5. Display results
    if not summary_df.empty:
        print("\n📊 File Summary:")
        print(summary_df[['City', 'File_Type', 'File_Size', 'Rows', 'Columns']])
        
        # Save to Excel
        output_path = os.path.join(folder_path, 'city_files_summary.xlsx')
        summary_df.to_excel(output_path, index=False)
        print(f"\n✅ Summary saved to: {output_path}")
    else:
        print("\n❌ All files failed to load. Check error messages above.")

Found Excel files in 'csv_files': ['csv_files\\bangalore_cars.xlsx', 'csv_files\\chennai_cars.xlsx', 'csv_files\\city_files_summary.xlsx', 'csv_files\\delhi_cars.xlsx', 'csv_files\\hyderabad_cars.xlsx', 'csv_files\\jaipur_cars.xlsx', 'csv_files\\kolkata_cars.xlsx']

📊 File Summary:
                 City     File_Type File_Size  Rows  Columns
0      bangalore_cars  Excel (XLSX)   0.55 MB  1481        5
1        chennai_cars  Excel (XLSX)   0.54 MB  1419        5
2  city_files_summary  Excel (XLSX)   0.01 MB     6        7
3          delhi_cars  Excel (XLSX)   0.56 MB  1485        5
4      hyderabad_cars  Excel (XLSX)   0.55 MB  1483        5
5         jaipur_cars  Excel (XLSX)    0.4 MB  1120        5
6        kolkata_cars  Excel (XLSX)   0.51 MB  1381        5

✅ Summary saved to: csv_files\city_files_summary.xlsx


In [2]:
# Test reading one file
test_file = city_files[0]  # or specify a filename
try:
    test_df = pd.read_excel(test_file)
    print(f"Successfully read {test_file}")
    print("Columns:", test_df.columns.tolist())
except Exception as e:
    print(f"Failed to read {test_file}: {str(e)}")

Successfully read csv_files\bangalore_cars.xlsx
Columns: ['new_car_detail', 'new_car_overview', 'new_car_feature', 'new_car_specs', 'car_links']


In [3]:
# Enhanced Version (Checks All Sheets):
# If your Excel files have multiple sheets, use this:

# Modified loop to check all sheets
for file in city_files:
    city_name = os.path.splitext(file)[0]
    try:
        xl = pd.ExcelFile(file)  # Open Excel file
        for sheet_name in xl.sheet_names:  # Loop through all sheets
            df_sample = xl.parse(sheet_name, nrows=2)
            file_size = f"{round(os.path.getsize(file)/(1024*1024), 2)} MB"
            summary_data.append({
                'City': city_name,
                'Sheet_Name': sheet_name,
                'File_Type': 'Excel (XLSX)',
                'File_Size': file_size,
                'Rows': xl.parse(sheet_name).shape[0],
                'Columns': xl.parse(sheet_name).shape[1],
                'Columns_List': df_sample.columns.tolist(),
                'Dtypes': df_sample.dtypes.to_dict()
            })
    except Exception as e:
        print(f"Error reading {file}: {str(e)}")

In [4]:

# Check for consistency across files:
# Compare columns across all cities
all_columns = summary_df['Columns_List'].explode().unique()
print("Unique columns across all files:", all_columns)

Unique columns across all files: ['new_car_detail' 'new_car_overview' 'new_car_feature' 'new_car_specs'
 'car_links' 'City' 'File_Type' 'File_Size' 'Rows' 'Columns'
 'Columns_List' 'Sample_First_Row']


In [5]:
# Merge all files into a single DataFrame:

all_cities = pd.concat(
    [pd.read_excel(file).assign(City=os.path.splitext(file)[0]) 
    for file in city_files
])

In [ ]:
#Reset the Index
all_cities = all_cities.reset_index(drop=True)

In [9]:
#Check for Missing Values
print(all_cities.isnull().sum())

new_car_detail         7
new_car_overview       7
new_car_feature        7
new_car_specs          7
car_links              7
City                   0
File_Type           8369
File_Size           8369
Rows                8369
Columns             8369
Columns_List        8369
Sample_First_Row    8369
dtype: int64


In [10]:
print(all_cities.head())
print(all_cities.info())

                                      new_car_detail  \
0  {'it': 0, 'ft': 'Petrol', 'bt': 'Hatchback', '...   
1  {'it': 0, 'ft': 'Petrol', 'bt': 'SUV', 'km': '...   
2  {'it': 0, 'ft': 'Petrol', 'bt': 'Hatchback', '...   
3  {'it': 0, 'ft': 'Petrol', 'bt': 'Sedan', 'km':...   
4  {'it': 0, 'ft': 'Diesel', 'bt': 'SUV', 'km': '...   

                                    new_car_overview  \
0  {'heading': 'Car overview', 'top': [{'key': 'R...   
1  {'heading': 'Car overview', 'top': [{'key': 'R...   
2  {'heading': 'Car overview', 'top': [{'key': 'R...   
3  {'heading': 'Car overview', 'top': [{'key': 'R...   
4  {'heading': 'Car overview', 'top': [{'key': 'R...   

                                     new_car_feature  \
0  {'heading': 'Features', 'top': [{'value': 'Pow...   
1  {'heading': 'Features', 'top': [{'value': 'Pow...   
2  {'heading': 'Features', 'top': [{'value': 'Pow...   
3  {'heading': 'Features', 'top': [{'value': 'Pow...   
4  {'heading': 'Features', 'top': [{'value': '

In [ ]:
all_cities.columns = all_cities.columns.str.strip().str.lower().str.replace(' ', '_')
#Makes column names consistent.

In [13]:
#Handle Data Type Issues  Convert columns to appropriate data types.
all_cities['date_column'] = pd.to_datetime(all_cities['date_column'])

KeyError: 'date_column'

In [14]:
print(all_cities.columns.tolist())

['new_car_detail', 'new_car_overview', 'new_car_feature', 'new_car_specs', 'car_links', 'city', 'file_type', 'file_size', 'rows', 'columns', 'columns_list', 'sample_first_row']


In [17]:
all_cities['date'] = pd.to_datetime(all_cities['date'], errors='coerce')  # converts errors to NaT

KeyError: 'date'

**1. Data Import and Structuring**

**a) Import and Concatenate City Datasets**

In [6]:
import pandas as pd
import os
import json
from glob import glob

def process_city_file(file_path, city_name):
    """Process a single city file and return structured DataFrame"""
    df = pd.read_csv(file_path)  # or pd.read_excel() if Excel files
    
    # Extract nested JSON data from columns
    for col in ['new_car_detail', 'new_car_overview', 'new_car_feature', 'new_car_specs']:
        df[col] = df[col].apply(lambda x: json.loads(x) if isinstance(x, str) else x)
    
    # Flatten nested JSON structures
    details = pd.json_normalize(df['new_car_detail'])
    overview = pd.json_normalize(df['new_car_overview'].apply(lambda x: x['top']))
    features = pd.json_normalize(df['new_car_feature'].apply(lambda x: [item['value'] for item in x['top']]))
    specs = pd.json_normalize(df['new_car_specs'].apply(lambda x: x['top']))
    
    # Combine all data
    processed_df = pd.concat([
        df[['car_links']],
        details,
        overview,
        features,
        specs
    ], axis=1)
    
    # Add city column
    processed_df['City'] = city_name
    
    return processed_df

def process_all_cities(data_dir):
    """Process all city files in directory"""
    all_cities = []
    
    # Get all city files (assuming naming pattern like 'Chennai.csv', 'Mumbai.csv')
    city_files = glob(os.path.join(data_dir, '*.csv'))  # or '*.xlsx' for Excel
    
    for file_path in city_files:
        city_name = os.path.basename(file_path).split('.')[0]
        city_df = process_city_file(file_path, city_name)
        all_cities.append(city_df)
    
    # Concatenate all city DataFrames
    combined_df = pd.concat(all_cities, ignore_index=True)
    
    return combined_df

# Usage:
data_directory = 'path/to/your/city/files'
combined_data = process_all_cities(data_directory)

ValueError: No objects to concatenate

**2. Handling Missing Values**

In [ ]:
def handle_missing_values(df):
    """Handle missing values in the dataset"""
    
    # Numerical columns - fill with median
    numerical_cols = ['km', 'modelYear', 'ownerNo', 'priceActual']
    for col in numerical_cols:
        if col in df.columns:
            # Clean numerical columns first (remove commas, non-numeric chars)
            df[col] = pd.to_numeric(df[col].astype(str).str.replace('[^\d.]', '', regex=True), errors='coerce')
            df[col].fillna(df[col].median(), inplace=True)
    
    # Categorical columns - fill with mode or 'Unknown'
    categorical_cols = ['ft', 'bt', 'transmission', 'owner', 'oem', 'model']
    for col in categorical_cols:
        if col in df.columns:
            df[col].fillna(df[col].mode()[0] if not df[col].mode().empty else 'Unknown', inplace=True)
    
    # Price handling - extract numerical value from string
    if 'price' in df.columns:
        df['price'] = df['price'].str.extract(r'₹\s*([\d,.]+)')[0]
        df['price'] = pd.to_numeric(df['price'].str.replace(',', ''), errors='coerce')
        df['price'].fillna(df['price'].median(), inplace=True)
    
    return df

# Apply missing value handling
cleaned_data = handle_missing_values(combined_data)

**Standardizing Data Formats**

In [ ]:
def standardize_data_formats(df):
    """Standardize data formats across columns"""
    
    # Convert kms driven to numeric
    if 'km' in df.columns:
        df['km'] = df['km'].astype(str).str.replace('Kms', '').str.replace(',', '').str.strip()
        df['km'] = pd.to_numeric(df['km'], errors='coerce')
    
    # Convert engine displacement to numeric (cc)
    if 'engineDisplacement' in df.columns:
        df['engineDisplacement_cc'] = df['engineDisplacement'].str.extract(r'(\d+)').astype(float)
    
    # Extract power values
    if 'maxPower' in df.columns:
        df['maxPower_bhp'] = df['maxPower'].str.extract(r'([\d.]+)').astype(float)
    
    # Convert torque to numeric
    if 'torque' in df.columns:
        df['torque_nm'] = df['torque'].str.extract(r'([\d.]+)').astype(float)
    
    # Convert year columns to datetime
    year_cols = ['modelYear', 'registrationYear']
    for col in year_cols:
        if col in df.columns:
            df[col] = pd.to_datetime(df[col], format='%Y', errors='coerce').dt.year
    
    # Standardize owner information
    if 'owner' in df.columns:
        df['owner'] = df['owner'].str.replace(r'\d(st|nd|rd|th)\s*Owner', lambda m: m.group(1) + ' Owner', regex=True)
    
    return df

# Apply standardization
standardized_data = standardize_data_formats(cleaned_data)

**Feature Engineering**

In [ ]:
def perform_feature_engineering(df):
    """Create new features from existing data"""
    
    # Calculate car age
    current_year = pd.Timestamp.now().year
    if 'modelYear' in df.columns:
        df['car_age'] = current_year - df['modelYear']
    
    # Create binary features from car features
    if 'features' in df.columns:
        all_features = set()
        for feature_list in df['features']:
            if isinstance(feature_list, list):
                all_features.update(feature_list)
        
        for feature in all_features:
            df[f'has_{feature}'] = df['features'].apply(lambda x: feature in x if isinstance(x, list) else False)
    
    # Create transmission type binary columns
    if 'transmission' in df.columns:
        df['is_automatic'] = df['transmission'].str.contains('Automatic', case=False)
        df['is_manual'] = df['transmission'].str.contains('Manual', case=False)
    
    return df

# Apply feature engineering
final_data = perform_feature_engineering(standardized_data)

**5. Saving Processed Data**

In [ ]:
# Save the processed data
final_data.to_csv('processed_car_data.csv', index=False)
# Or save to pickle for preserving data types
final_data.to_pickle('processed_car_data.pkl')

**4. Encoding Categorical Variables**

In [ ]:
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer

def encode_categorical_variables(df):
    """Encode categorical variables using appropriate techniques"""
    
    # Identify categorical columns
    nominal_cats = ['oem', 'model', 'City', 'fuelType', 'transmissionType']  # No inherent ordering
    ordinal_cats = ['owner']  # Has meaningful order (1st > 2nd > 3rd Owner)
    
    # One-hot encoding for nominal variables
    onehot_encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
    onehot_encoded = onehot_encoder.fit_transform(df[nominal_cats])
    
    # Create DataFrame from one-hot encoded data
    onehot_df = pd.DataFrame(
        onehot_encoded,
        columns=onehot_encoder.get_feature_names_out(nominal_cats)
    )
    
    # Ordinal encoding for ordered categories
    owner_order = ['4th Owner', '3rd Owner', '2nd Owner', '1st Owner']  # Lower is better
    ordinal_encoder = OrdinalEncoder(categories=[owner_order])
    df['owner_encoded'] = ordinal_encoder.fit_transform(df[['owner']])
    
    # Label encoding for binary categories (if any)
    binary_cols = ['is_automatic', 'is_manual']  # Example binary columns
    for col in binary_cols:
        if col in df.columns:
            df[col] = LabelEncoder().fit_transform(df[col])
    
    # Combine encoded data with original numerical data
    numerical_cols = df.select_dtypes(include=['int64', 'float64']).columns
    final_df = pd.concat([
        df[numerical_cols],
        onehot_df,
        df[['owner_encoded']]
    ], axis=1)
    
    return final_df, onehot_encoder, ordinal_encoder

# Apply encoding
encoded_data, ohe, ord_enc = encode_categorical_variables(final_data)

**5. Normalizing Numerical Features**

In [ ]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

def normalize_numerical_features(df, scaling_method='minmax'):
    """Normalize numerical features to common scale"""
    
    # Select numerical columns (excluding target and encoded columns)
    numerical_cols = [col for col in df.columns 
                    if df[col].dtype in ['int64', 'float64'] 
                    and col != 'price'  # Exclude target variable
                    and not col.endswith('_encoded')]
    
    # Apply selected scaling method
    if scaling_method == 'minmax':
        scaler = MinMaxScaler()
    else:  # standard
        scaler = StandardScaler()
    
    df[numerical_cols] = scaler.fit_transform(df[numerical_cols])
    
    return df, scaler

# Apply normalization (choose either 'minmax' or 'standard')
normalized_data, feature_scaler = normalize_numerical_features(encoded_data, 'minmax')

**6. Removing Outliers**

In [ ]:
from scipy import stats

def handle_outliers(df, method='iqr', threshold=1.5):
    """Identify and handle outliers in numerical features"""
    
    numerical_cols = df.select_dtypes(include=['int64', 'float64']).columns
    
    if method == 'iqr':
        # IQR method
        for col in numerical_cols:
            if col != 'price':  # Don't remove based on target variable
                Q1 = df[col].quantile(0.25)
                Q3 = df[col].quantile(0.75)
                IQR = Q3 - Q1
                lower_bound = Q1 - threshold * IQR
                upper_bound = Q3 + threshold * IQR
                
                # Cap outliers instead of removing
                df[col] = df[col].clip(lower_bound, upper_bound)
    
    elif method == 'zscore':
        # Z-score method
        for col in numerical_cols:
            if col != 'price':
                z_scores = stats.zscore(df[col])
                df = df[(np.abs(z_scores) < threshold)]
    
    return df

# Apply outlier handling (choose either 'iqr' or 'zscore')
cleaned_data = handle_outliers(normalized_data, 'iqr', 1.5)

**Complete Data Processing Pipeline**

In [ ]:
def full_data_processing_pipeline(data_dir, city_files):
    """Complete data processing pipeline"""
    
    # 1. Import and concatenate city datasets
    combined_data = process_all_cities(data_dir, city_files)
    
    # 2. Handle missing values
    cleaned_data = handle_missing_values(combined_data)
    
    # 3. Standardize data formats
    standardized_data = standardize_data_formats(cleaned_data)
    
    # 4. Feature engineering
    final_data = perform_feature_engineering(standardized_data)
    
    # 5. Encode categorical variables
    encoded_data, ohe, ord_enc = encode_categorical_variables(final_data)
    
    # 6. Normalize numerical features
    normalized_data, feature_scaler = normalize_numerical_features(encoded_data)
    
    # 7. Handle outliers
    processed_data = handle_outliers(normalized_data)
    
    return processed_data, ohe, ord_enc, feature_scaler

# Example usage:
data_dir = 'path/to/city/files'
city_files = ['Chennai.csv', 'Mumbai.csv', 'Delhi.csv']  # etc.
final_processed_data, ohe, ord_enc, scaler = full_data_processing_pipeline(data_dir, city_files)